## Building Dynamic AI Assistants with Amazon Bedrock Inline Agents

In this notebook, we'll walk through the process of setting up and invoking an inline agent, showcasing its flexibility and power in creating dynamic AI assistants. By following our progressive approach, you will gain a comprehensive understanding of how to use inline agents for various use cases and complexity levels. Throughout a single interactive conversation, we will demonstrate how the agent can be enhanced `on the fly` with new tools and instructions while maintaining context of our ongoing discussion.

We'll follow a progressive approach to building our assistant:

1. Simple Inline Agent: We'll start with a basic inline agent with a code interpreter.
2. Adding Knowledge Bases: We'll enhance our agent by incorporating a knowledge base with role-based access.
3. Integrating Action Groups: Finally, we'll add custom tools to extend the agent's functionality.

## What are Inline Agents?

[Inline agents](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-create-inline.html) are a powerful feature of Amazon Bedrock that allow developers to create flexible and adaptable AI assistants. 

Unlike traditional static agents, inline agents can be dynamically configured at runtime, enabling real time adjustments to their behavior, capabilities, and knowledge base.

Key features of inline agents include:

1. **Dynamic configuration**: Modify the agent's instructions, action groups, and other parameters on the fly.
2. **Flexible integration**: Easily incorporate external APIs and services as needed for each interaction.
3. **Contextual adaptation**: Adjust the agent's responses based on user roles, preferences, or specific scenarios.

## Why Use Inline Agents?

Inline agents offer several advantages for building AI applications:

1. **Rapid prototyping**: Quickly experiment with different configurations without redeploying your application.
2. **Personalization**: Tailor the agent's capabilities to individual users or use cases in real time.
3. **Scalability**: Efficiently manage a single agent that can adapt to multiple roles or functions.
4. **Cost effectiveness**: Optimize resource usage by dynamically selecting only the necessary tools and knowledge for each interaction.

## Prerequisites

Before you begin, make sure that you have:

1. An active AWS account with access to Amazon Bedrock.
2. Necessary permissions to create and invoke inline agents.
3. Be sure to complete additonal prerequisites, visit [Amazon Bedrock Inline Agent prerequisites documentation](https://docs.aws.amazon.com/bedrock/latest/userguide/inline-agent-prereq.html) to learn more.

### Installing prerequisites
Let's begin with installing the required packages. This step is important as you need `boto3` version `1.35.68` or later to use inline agents.

In [1]:
# uncomment to install the required python packages
!pip install --upgrade -r requirements.txt


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
# # restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

## Setup and Imports

First, let's import the necessary libraries and set up our Bedrock client.

In [3]:
import os
import json
from pprint import pprint
import boto3
from datetime import datetime
import random
import pprint
from termcolor import colored
from rich.console import Console
from rich.markdown import Markdown
import ipywidgets as widgets

session = boto3.session.Session()
region = session.region_name

# Runtime Endpoints
bedrock_rt_client = boto3.client(
    "bedrock-agent-runtime",
    region_name=region
)

sts_client = boto3.client("sts")
account_id = sts_client.get_caller_identity()["Account"]

# To manage session id:
random_int = random.randint(1,100000)

### Model Selection
Select the Foundation model to use.

In [31]:
#Create a dropdown for model selection
model_dropdown = widgets.Dropdown(
    options=[
        ('Claude 3.0 Sonnet', 'anthropic.claude-3-sonnet-20240229-v1:0'),
        ('Claude 3.5 Sonnet', 'anthropic.claude-3-5-sonnet-20240620-v1:0'),
        # ('Claude 3.7 Sonnet', 'anthropic.claude-3-7-sonnet-20250219-v1:0')
    ],
    description='Model:',
    disabled=False,
)

display(model_dropdown)

Dropdown(description='Model:', options=(('Claude 3.0 Sonnet', 'anthropic.claude-3-sonnet-20240229-v1:0'), ('Cl…

## Configuring the Inline Agent

Next, we'll set up the basic configuration for our Amazon Bedrock Inline Agent. This includes specifying the foundation model, session management, and basic instructions.

In [6]:
# change model id as needed:
model_id = model_dropdown.value

sessionId = f'custom-session-id-{random_int}'
endSession = False
enableTrace = True

# customize instructions of inline agent:
agent_instruction = """You are a helpful AI assistant helping Octank Inc employees with their questions and processes. 
You write short and direct responses while being cheerful. You have access to python coding environment that helps you extend your capabilities."""

## Basic Inline Agent Invocation

Let's start by invoking a simple inline agent with just the foundation model and basic instructions.

In [7]:
# prepare request parameters before invoking inline agent
request_params = {
    "instruction": agent_instruction,
    "foundationModel": model_id,
    "sessionId": sessionId,
    "endSession": endSession,
    "enableTrace": enableTrace,
}

# define code interpreter tool
code_interpreter_tool = {
    "actionGroupName": "UserInputAction",
    "parentActionGroupSignature": "AMAZON.CodeInterpreter",
    "toolChoice": "auto"
}

# add the tool to request parameter of inline agent
request_params["actionGroups"] = [code_interpreter_tool]

# enable traces
request_params["enableTrace"] = True

In [8]:
# enter the question you want the inline agent to answer
request_params['inputText'] = 'what is the time right now in pacific timezone?'

### Compatibility

In [9]:
def process_tool_response(response_event):
    """
    Process tool response events to ensure tool_use_id is properly included
    for Claude 3.5 Sonnet compatibility
    """
    if "trace" in response_event and "trace" in response_event["trace"]:
        trace_data = response_event["trace"]["trace"]

        # Check for orchestration trace with tool interactions
        if "orchestrationTrace" in trace_data:
            orch_trace = trace_data["orchestrationTrace"]

            # If we have tool invocation and observation pairs, ensure IDs are linked
            if "invocationInput" in orch_trace and "observation" in orch_trace:
                tool_input = orch_trace.get("invocationInput", {})
                observation = orch_trace.get("observation", {})

                # Extract tool_use_id from input if present
                tool_use_id = None
                if "actionGroupInvocationInput" in tool_input:
                    tool_use_id = tool_input["actionGroupInvocationInput"].get("id")
                elif "codeInterpreterInvocationInput" in tool_input:
                    tool_use_id = tool_input["codeInterpreterInvocationInput"].get("id")

                # If we have a tool_use_id and an observation without one, add it
                if tool_use_id:
                    if "actionGroupInvocationOutput" in observation:
                        if "id" not in observation["actionGroupInvocationOutput"]:
                            observation["actionGroupInvocationOutput"]["id"] = tool_use_id
                    elif "codeInterpreterInvocationOutput" in observation:
                        if "id" not in observation["codeInterpreterInvocationOutput"]:
                            observation["codeInterpreterInvocationOutput"]["id"] = tool_use_id

    return response_event

### Invoking a simple Inline Agent

We'll send a request to the agent asking it to perform a simple calculation or code execution task. This will showcase how the agent can interpret and run code on the fly.

To do so, we will use the [InvokeInlineAgent](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_agent-runtime_InvokeInlineAgent.html) API via boto3 `bedrock-agent-runtime` client.

Our function `invoke_inline_agent_helper` also helps us processing the agent trace request and format it for easier readibility. You do not have to use this function in your system, but it will make it easier to observe the code used by code interpreter, the function invocations and the knowledge base content.

We also provide the metrics for the agent invocation time and the input and output tokens

In [17]:
def invoke_inline_agent_helper(client, request_params, trace_level="core"):
    # Check if using Claude 3.5
    is_claude_3 = "claude-3-sonnet" in model_id

    if is_claude_3:
        for i in request_params["actionGroups"]:
            i.pop('toolChoice',None)
    else:
        # Ensure we include sessionState to maintain context for tools
        if "sessionState" not in request_params:
            request_params["sessionState"] = "{}"

    _time_before_call = datetime.now()

    _agent_resp = client.invoke_inline_agent(
        **request_params
    )

    if request_params["enableTrace"]:
        if trace_level == "all":
            print(f"invokeAgent API response object: {_agent_resp}")
        else:
            print(
                f"invokeAgent API request ID: {_agent_resp['ResponseMetadata']['RequestId']}"
            )
            session_id = request_params["sessionId"]
            print(f"invokeAgent API session ID: {session_id}")

    # Return error message if invoke was unsuccessful
    if _agent_resp["ResponseMetadata"]["HTTPStatusCode"] != 200:
        _error_message = f"API Response was not 200: {_agent_resp}"
        if request_params["enableTrace"] and trace_level == "all":
            print(_error_message)
        return _error_message

    _total_in_tokens = 0
    _total_out_tokens = 0
    _total_llm_calls = 0
    _orch_step = 0
    _sub_step = 0
    _trace_truncation_lenght = 300
    _time_before_orchestration = datetime.now()

    _agent_answer = ""
    _event_stream = _agent_resp["completion"]

    try:
        for _event in _event_stream:
            # Process the event to ensure tool_use_id compatibility
            _event = process_tool_response(_event)
            _sub_agent_alias_id = None

            if "chunk" in _event:
                _data = _event["chunk"]["bytes"]
                _agent_answer = _data.decode("utf8")

            if "trace" in _event and request_params["enableTrace"]:
                if "failureTrace" in _event["trace"]["trace"]:
                    print(
                        colored(
                            f"Agent error: {_event['trace']['trace']['failureTrace']['failureReason']}",
                            "red",
                        )
                    )

                if "orchestrationTrace" in _event["trace"]["trace"]:
                    _orch = _event["trace"]["trace"]["orchestrationTrace"]

                    if trace_level in ["core", "outline"]:
                        if "rationale" in _orch:
                            _rationale = _orch["rationale"]
                            print(colored(f"{_rationale['text']}", "blue"))

                        if "invocationInput" in _orch:
                            # NOTE: when agent determines invocations should happen in parallel
                            # the trace objects for invocation input still come back one at a time.
                            _input = _orch["invocationInput"]
                            print(_input)

                            if "actionGroupInvocationInput" in _input:
                                if 'function' in _input['actionGroupInvocationInput']:
                                    tool = _input['actionGroupInvocationInput']['function']
                                elif 'apiPath' in _input['actionGroupInvocationInput']:
                                    tool = _input['actionGroupInvocationInput']['apiPath']
                                else:
                                    tool = 'undefined'
                                if trace_level == "outline":
                                    print(
                                        colored(
                                            f"Using tool: {tool}",
                                            "magenta",
                                        )
                                    )
                                else:
                                    print(
                                        colored(
                                            f"Using tool: {tool} with these inputs:",
                                            "magenta",
                                        )
                                    )
                                    if (
                                        len(
                                            _input["actionGroupInvocationInput"][
                                                "parameters"
                                            ]
                                        )
                                        == 1
                                    ) and (
                                        _input["actionGroupInvocationInput"][
                                            "parameters"
                                        ][0]["name"]
                                        == "input_text"
                                    ):
                                        print(
                                            colored(
                                                f"{_input['actionGroupInvocationInput']['parameters'][0]['value']}",
                                                "magenta",
                                            )
                                        )
                                    else:
                                        print(
                                            colored(
                                                f"{_input['actionGroupInvocationInput']['parameters']}\n",
                                                "magenta",
                                            )
                                        )

                            elif "codeInterpreterInvocationInput" in _input:
                                if trace_level == "outline":
                                    print(
                                        colored(
                                            f"Using code interpreter", "magenta"
                                        )
                                    )
                                else:
                                    console = Console()
                                    _gen_code = _input[
                                        "codeInterpreterInvocationInput"
                                    ]["code"]
                                    _code = f"```python\n{_gen_code}\n```"

                                    console.print(
                                        Markdown(f"**Generated code**\n{_code}")
                                    )

                        if "observation" in _orch:
                            if trace_level == "core":
                                _output = _orch["observation"]
                                if "actionGroupInvocationOutput" in _output:
                                    print(
                                        colored(
                                            f"--tool outputs:\n{_output['actionGroupInvocationOutput']['text'][0:_trace_truncation_lenght]}...\n",
                                            "magenta",
                                        )
                                    )

                                if "agentCollaboratorInvocationOutput" in _output:
                                    _collab_name = _output[
                                        "agentCollaboratorInvocationOutput"
                                    ]["agentCollaboratorName"]
                                    _collab_output_text = _output[
                                        "agentCollaboratorInvocationOutput"
                                    ]["output"]["text"][0:_trace_truncation_lenght]
                                    print(
                                        colored(
                                            f"\n----sub-agent {_collab_name} output text:\n{_collab_output_text}...\n",
                                            "magenta",
                                        )
                                    )

                                if "finalResponse" in _output:
                                    print(
                                        colored(
                                            f"Final response:\n{_output['finalResponse']['text'][0:_trace_truncation_lenght]}...",
                                            "cyan",
                                        )
                                    )


                    if "modelInvocationOutput" in _orch:
                        _orch_step += 1
                        _sub_step = 0
                        print(colored(f"---- Step {_orch_step} ----", "green"))

                        _llm_usage = _orch["modelInvocationOutput"]["metadata"][
                            "usage"
                        ]
                        _in_tokens = _llm_usage.get("inputTokens",None)
                        _total_in_tokens += _in_tokens

                        _out_tokens = _llm_usage["outputTokens"]
                        _total_out_tokens += _out_tokens

                        _total_llm_calls += 1
                        _orch_duration = (
                            datetime.now() - _time_before_orchestration
                        )

                        print(
                            colored(
                                f"Took {_orch_duration.total_seconds():,.1f}s, using {_in_tokens+_out_tokens} tokens (in: {_in_tokens}, out: {_out_tokens}) to complete prior action, observe, orchestrate.",
                                "yellow",
                            )
                        )

                        # restart the clock for next step/sub-step
                        _time_before_orchestration = datetime.now()

                elif "preProcessingTrace" in _event["trace"]["trace"]:
                    _pre = _event["trace"]["trace"]["preProcessingTrace"]
                    if "modelInvocationOutput" in _pre:
                        _llm_usage = _pre["modelInvocationOutput"]["metadata"][
                            "usage"
                        ]
                        _in_tokens = _llm_usage["inputTokens"]
                        _total_in_tokens += _in_tokens

                        _out_tokens = _llm_usage["outputTokens"]
                        _total_out_tokens += _out_tokens

                        _total_llm_calls += 1

                        print(
                            colored(
                                "Pre-processing trace, agent came up with an initial plan.",
                                "yellow",
                            )
                        )
                        print(
                            colored(
                                f"Used LLM tokens, in: {_in_tokens}, out: {_out_tokens}",
                                "yellow",
                            )
                        )

                elif "postProcessingTrace" in _event["trace"]["trace"]:
                    _post = _event["trace"]["trace"]["postProcessingTrace"]
                    if "modelInvocationOutput" in _post:
                        _llm_usage = _post["modelInvocationOutput"]["metadata"][
                            "usage"
                        ]
                        _in_tokens = _llm_usage["inputTokens"]
                        _total_in_tokens += _in_tokens

                        _out_tokens = _llm_usage["outputTokens"]
                        _total_out_tokens += _out_tokens

                        _total_llm_calls += 1
                        print(colored("Agent post-processing complete.", "yellow"))
                        print(
                            colored(
                                f"Used LLM tokens, in: {_in_tokens}, out: {_out_tokens}",
                                "yellow",
                            )
                        )

                if trace_level == "all":
                    print(json.dumps(_event["trace"], indent=2))

            if "files" in _event.keys() and request_params["enableTrace"]:
                console = Console()
                files_event = _event["files"]
                console.print(Markdown("**Files**"))

                files_list = files_event["files"]
                for this_file in files_list:
                    print(f"{this_file['name']} ({this_file['type']})")
                    file_bytes = this_file["bytes"]

                    # save bytes to file, given the name of file and the bytes
                    file_name = os.path.join("output", this_file["name"])
                    with open(file_name, "wb") as f:
                        f.write(file_bytes)

        if request_params["enableTrace"]:
            duration = datetime.now() - _time_before_call

            if trace_level in ["core", "outline"]:
                print(
                    colored(
                        f"Agent made a total of {_total_llm_calls} LLM calls, "
                        + f"using {_total_in_tokens+_total_out_tokens} tokens "
                        + f"(in: {_total_in_tokens}, out: {_total_out_tokens})"
                        + f", and took {duration.total_seconds():,.1f} total seconds",
                        "yellow",
                    )
                )

            if trace_level == "all":
                print(f"Returning agent answer as: {_agent_answer}")

        return _agent_answer

    except Exception as e:
        print(f"Caught exception while processing input to invokeAgent:\n")
        input_text = request_params["inputText"]
        print(f"  for input text:\n{input_text}\n")
        print(
            f"  request ID: {_agent_resp['ResponseMetadata']['RequestId']}, retries: {_agent_resp['ResponseMetadata']['RetryAttempts']}\n"
        )
        print(f"Error: {e}")
        raise Exception("Unexpected exception: ", e)

In [18]:
invoke_inline_agent_helper(bedrock_rt_client, request_params, trace_level="core")

invokeAgent API request ID: c51ab19c-a590-424c-b937-d973045afdf8
invokeAgent API session ID: custom-session-id-14904
---- Step 1 ----
Took 3.1s, using 2337 tokens (in: 2231, out: 106) to complete prior action, observe, orchestrate.
Since the knowledge base did not contain information about employee compensation bonuses, I will need to apologize that I could not find a specific answer to this question.
Final response:
I'm sorry, I could not find any details about the employee compensation bonus amounts or policies in the company knowledge base that was provided to me. The knowledge base seems to be missing information specifically related to bonus compensation. I do not have enough information to accurately answe...
Agent made a total of 1 LLM calls, using 2337 tokens (in: 2231, out: 106), and took 3.4 total seconds


"I'm sorry, I could not find any details about the employee compensation bonus amounts or policies in the company knowledge base that was provided to me. The knowledge base seems to be missing information specifically related to bonus compensation. I do not have enough information to accurately answer how much the employee compensation bonus is."

## Adding a Knowledge Base

Now, we'll demonstrate how to incorporate a knowledge base into our inline agent invocation. Let's first create a knowledge base using fictional HR policy documents that we will later use in with inline agent.

We will use [Amazon Bedrock Knowledge Base](https://aws.amazon.com/bedrock/knowledge-bases/) to create our knowledge base. To do so, we use the support function `create_knowledge_base` available in the `create_knowledge_base.py` file. It will abstract away the work to create the underline vector database, the vector indexes with the appropriated chunking strategy as well as the indexation of the documents to the knowledge base. Take a look at the `create_knowledge_base.py` file for more details.

In [19]:
import os
from create_knowledge_base import create_knowledge_base

# Configuration
bucket_name = f"inline-agent-bucket-{random_int}"
kb_name = f"policy-kb-{random_int}"
data_path = "policy_documents"

# Create knowledge base and upload documents
kb_id, bucket_name, kb_metadata = create_knowledge_base(region, bucket_name, kb_name, data_path)

### Setting up Knowledge Base configuration to invoke inline agent

Let's now set up the knowledge base configuration to invoke our inline agent

In [21]:
# define number of chunks to retrieve
num_results = 3
search_strategy = "HYBRID"

# provide instructions about knowledge base that inline agent can use
kb_description = 'This knowledge base contains information about company HR policies, code or conduct, performance reviews and much more'

# lets define access level for metadata filtering
user_profile = 'basic'
access_filter = {
    "equals": {
        "key": "access_level",
        "value": user_profile
    }
}

# lets revise our Knowledge bases configuration
kb_config = {
    "knowledgeBaseId": kb_id,
    "description": kb_description,
    "retrievalConfiguration": {
        "vectorSearchConfiguration": {
            "filter": access_filter,
            "numberOfResults": num_results,
            "overrideSearchType": "HYBRID"
        }
    }
}

# lets add knowledge bases to our request parameters
request_params["knowledgeBases"] = [kb_config]
    
# update the agent instructions to inform inline agent that it has access to a knowlegde base
new_capabilities = """You have access to Octank Inc company policies knowledge base. 
Use this database to search for information about company policies, company HR policies, code or conduct, performance reviews and much more. And use them to briefly answer the use question."""
request_params["instruction"] += f"\n\n{new_capabilities}"

# check updated request parameters including instructions for the inline agent
print(request_params)

{'instruction': 'You are a helpful AI assistant helping Octank Inc employees with their questions and processes. \nYou write short and direct responses while being cheerful. You have access to python coding environment that helps you extend your capabilities.\n\nYou have access to Octank Inc company policies knowledge base. \nUse this database to search for information about company policies, company HR policies, code or conduct, performance reviews and much more. And use them to briefly answer the use question.\n\nYou have access to Octank Inc company policies knowledge base. \nUse this database to search for information about company policies, company HR policies, code or conduct, performance reviews and much more. And use them to briefly answer the use question.', 'foundationModel': 'anthropic.claude-3-sonnet-20240229-v1:0', 'sessionId': 'custom-session-id-14904', 'endSession': False, 'enableTrace': True, 'actionGroups': [{'actionGroupName': 'UserInputAction', 'parentActionGroupSign

### Querying the Enhanced Agent

We'll send a query that requires the agent to retrieve information from the knowledge base and provide an informed response.

In [22]:
# enter the question that will use knowledge bases
request_params['inputText'] = 'How much is the employee compensation bonus?'

In [24]:
# invoke the inline agent
invoke_inline_agent_helper(bedrock_rt_client, request_params)

invokeAgent API request ID: e9ad2098-c9e9-47ec-931f-c67fd01a558c
invokeAgent API session ID: custom-session-id-14904
---- Step 1 ----
Took 4.2s, using 2706 tokens (in: 2549, out: 157) to complete prior action, observe, orchestrate.
I have already searched the provided knowledge base and stated multiple times that I could not find information to answer this specific question about employee compensation bonus amounts. Repeating the same response will not provide any new information, so I should acknowledge that I cannot provide a satisfactory answer.
Final response:
I'm afraid I do not have enough information in the company knowledge base provided to me to specify the amount of employee compensation bonuses at Octank Inc. I have searched thoroughly but could not find any details related to bonus compensation policies or amounts. Without access to that data, I c...
Agent made a total of 1 LLM calls, using 2706 tokens (in: 2549, out: 157), and took 4.3 total seconds


"I'm afraid I do not have enough information in the company knowledge base provided to me to specify the amount of employee compensation bonuses at Octank Inc. I have searched thoroughly but could not find any details related to bonus compensation policies or amounts. Without access to that data, I cannot give you a definitive answer to your question about how much the employee compensation bonus is. I apologize that I do not have a more complete response to provide."

### Analyzing the Knowledge Base Integration

We see that there are two types of access levels defined in the knowledge base, basic and manager. Compensation related access is `Manager` only. Let's try the same query with proper filter.

In [25]:
# lets define access level for metadata filtering
user_profile = 'Manager'
# user_profile = 'basic'
access_filter = {
    "equals": {
        "key": "access_level",
        "value": user_profile
    }
}

# lets revise our Knowledge bases configuration
kb_config = {
    "knowledgeBaseId": kb_id,
    "description": kb_description,
    "retrievalConfiguration": {
        "vectorSearchConfiguration": {
            "filter": access_filter,
            "numberOfResults": num_results,
            "overrideSearchType": "HYBRID"
        }
    }
}

# lets add knowledge bases to our request parameters
request_params["knowledgeBases"] = [kb_config]

# invoke the inline agent
invoke_inline_agent_helper(bedrock_rt_client, request_params)

invokeAgent API request ID: fd7f3a85-70cd-4380-a493-d63da5c7f615
invokeAgent API session ID: custom-session-id-14904
---- Step 1 ----
Took 9.6s, using 2929 tokens (in: 2722, out: 207) to complete prior action, observe, orchestrate.
I have clearly stated multiple times now that I do not have access to information about employee compensation bonus amounts in the provided company knowledge base. Repeating myself further will not provide any new or useful information to the user. I should acknowledge the limitation of my knowledge and suggest the user consult official company policies or human resources if they need a definitive answer.
Final response:
I'm sorry, but as I've mentioned, I do not have any information about the specific amounts for employee compensation bonuses at Octank Inc. in the knowledge base I can access. I've searched thoroughly but could not find details on bonus compensation policies or amounts. Without being able to referen...
Agent made a total of 1 LLM calls, usin

"I'm sorry, but as I've mentioned, I do not have any information about the specific amounts for employee compensation bonuses at Octank Inc. in the knowledge base I can access. I've searched thoroughly but could not find details on bonus compensation policies or amounts. Without being able to reference an official company policy on this topic, I cannot provide a definitive answer to your question. If you need to know the exact bonus compensation details, I would suggest consulting Octank's official policies or human resources department directly. I apologize that I do not have more complete information to share regarding this particular query."

## Integrating Action Groups

In this section, we'll show how to add a custom tool (action group) to our agent invocation. This illustrates how to extend the agent's functionality with external services via the API.

Let's first create a lambda function that we will later use in with inline agent.

In [26]:
# run lambda function creation
from lambda_creator import create_lambda_function_and_its_resources
import os

present_directory = os.getcwd()
lambda_function_code_path = str(present_directory) + "/pto_lambda/lambda_function.py"

# Create all resources
resources = create_lambda_function_and_its_resources(
    region=region,
    account_id=account_id,
    custom_name=f"hr-inlineagent-lambda-{random_int}",
    lambda_code_path=lambda_function_code_path
)

# Access the created resources
lambda_function = resources['lambda_function']
lambda_function_arn = lambda_function['FunctionArn']
print(lambda_function_arn)

arn:aws:lambda:us-east-1:537124940578:function:hr-inlineagent-lambda-14904-us-east-1-537124940578


### Configuring the Agent with the Action Group

We'll update our agent configuration to include the new action group, allowing it to interact with the external service.
For this example we are providing an OpenAPI Schema to define our action group tools. You can also use function definition to do the same, but your lambda function even will change a bit. For more information see the documentation [here](https://docs.aws.amazon.com/bedrock/latest/userguide/action-define.html)

In [27]:
apply_vacation_tool = {
            'actionGroupName': 'FetchDetails',
            "actionGroupExecutor": {
                "lambda": lambda_function_arn
            }, "apiSchema": {
                "payload": """
    {
    "openapi": "3.0.0",
    "info": {
        "title": "Vacation Management API",
        "version": "1.0.0",
        "description": "API for managing vacation requests"
    },
    "paths": {
        "/vacation": {
            "post": {
                "summary": "Process vacation request",
                "description": "Process a vacation request or check balance",
                "operationId": "processVacation",
                "parameters": [
                    {
                        "name": "action",
                        "in": "query",
                        "description": "The type of vacation action to perform",
                        "required": true,
                        "schema": {
                            "type": "string",
                            "enum": ["check_balance", "check balance", "apply", "request"],
                            "description": "Action type for vacation management"
                        }
                    },
                    {
                        "name": "days",
                        "in": "query",
                        "description": "Number of vacation days requested",
                        "required": false,
                        "schema": {
                            "type": "integer",
                            "minimum": 1
                        }
                    }
                ],
                "responses": {
                    "200": {
                        "description": "Request processed successfully",
                        "content": {
                            "application/json": {
                                "schema": {
                                    "type": "object",
                                    "properties": {
                                        "status": {
                                            "type": "string",
                                            "enum": ["approved", "pending", "rejected", "info"],
                                            "description": "Status of the vacation request"
                                        },
                                        "message": {
                                            "type": "string",
                                            "description": "Detailed response message"
                                        },
                                        "ticket_url": {
                                            "type": "string",
                                            "description": "Ticket URL for long vacation requests"
                                        }
                                    },
                                    "required": ["status", "message"]
                                }
                            }
                        }
                    }
                }
            }
        }
    }
}
    """
            },
            "description": "Process vacation and check leave balance",
            "toolChoice": "auto"
}
            
# update the tools that inline agent has access to
request_params["actionGroups"] = [code_interpreter_tool, apply_vacation_tool]

### Testing the Full Featured Agent

We'll send a complex query that requires the agent to use its language understanding, access the knowledge base, and interact with the external service via the action group.

### Analyzing the Complete Agent Behavior

We'll examine the agent's response, focusing on how it orchestrates different capabilities (language model, knowledge base, and external actions) to handle complex queries.

In [28]:
# ask question:
request_params['inputText'] = 'I will be out of office from 2024/11/28 for the next 3 days'

# invoke the inline agent
invoke_inline_agent_helper(bedrock_rt_client, request_params)

invokeAgent API request ID: ef41b71f-6801-4b10-bda8-293ecc119ede
invokeAgent API session ID: custom-session-id-14904
---- Step 1 ----
Took 8.8s, using 3238 tokens (in: 3146, out: 92) to complete prior action, observe, orchestrate.
To process this vacation request, I will need to invoke the processVacation tool with the appropriate parameters.
{'actionGroupInvocationInput': {'actionGroupName': 'FetchDetails', 'apiPath': '/vacation', 'executionType': 'LAMBDA', 'parameters': [{'name': 'action', 'type': 'string', 'value': 'request'}, {'name': 'days', 'type': 'integer', 'value': '3'}], 'verb': 'post'}, 'invocationType': 'ACTION_GROUP', 'traceId': 'ef41b71f-6801-4b10-bda8-293ecc119ede-0'}
Using tool: /vacation with these inputs:
[{'name': 'action', 'type': 'string', 'value': 'request'}, {'name': 'days', 'type': 'integer', 'value': '3'}]

--tool outputs:
{"status": "approved", "message": "Your 3-day leave request is approved. New balance: 22 days."}...

---- Step 2 ----
Took 2.1s, using 3367 

'Your 3-day vacation request from 2024/11/28 has been approved. You now have 22 days remaining in your vacation balance.'

## Clean up

Let's delete the resources that were created in this notebook

In [ ]:
lambda_client = boto3.client('lambda')
iam_client = boto3.client('iam')

def delete_iam_roles_and_policies(role_name, iam_client):
    try:
        iam_client.get_role(RoleName=role_name)
    except iam_client.exceptions.NoSuchEntityException:
        print(f"Role {role_name} does not exist") 
    attached_policies = iam_client.list_attached_role_policies(RoleName=role_name)["AttachedPolicies"]
    print(f"======Attached policies with role {role_name}========\n", attached_policies)
    for attached_policy in attached_policies:
        policy_arn = attached_policy["PolicyArn"]
        policy_name = attached_policy["PolicyName"]
        iam_client.detach_role_policy(RoleName=role_name, PolicyArn=policy_arn)
        print(f"Detached policy {policy_name} from role {role_name}")
        if str(policy_arn.split("/")[1]) == "service-role":
            print(f"Skipping deletion of service-linked role policy {policy_name}")
        else: 
            iam_client.delete_policy(PolicyArn=policy_arn)
            print(f"Deleted policy {policy_name} from role {role_name}")

    iam_client.delete_role(RoleName=role_name)
    print(f"Deleted role {role_name}")
    print("======== All IAM roles and policies deleted =========")
    
# delete lambda function
response = lambda_client.delete_function(
    FunctionName=resources['lambda_function']['FunctionName']
)
# delete lamnda role and policy
delete_iam_roles_and_policies(resources['lambda_role']['Role']['RoleName'], iam_client)
# delete knowledge base
kb_metadata.delete_kb(delete_s3_bucket=True, delete_iam_roles_and_policies=True)

## Conclusion

This notebook has demonstrated the key aspects of using the Amazon Bedrock Inline Agents API:

1. Basic agent invocation
2. Incorporating knowledge bases
3. Adding custom action groups
4. Implementing guardrails

By leveraging these API capabilities, developers can create dynamic, adaptable AI assistants that can be easily customized for various use cases without redeploying applications.

Key takeaways:
1. Inline agents offer great flexibility through their API
2. Knowledge bases and action groups can be easily integrated
3. Guardrails help maintain responsible AI practices